In [ ]:
# adding path to user defined modules
import sys
sys.path.append('../')

# importing modules
import tqdm
import joblib
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

If a user is new to reading books.<br> We can show them most popular books.


    1. Simple recommendation system

In [ ]:
# reading the csv file
mer_rat_book = pd.read_csv('../data/processed/mer_rat_book.csv.gz')

In [ ]:
# lets copy a data frame named popular_books
popular_books= mer_rat_book[['ISBN','Book-Rating','Total-Reviewers']].copy()

# multiplying ratings with count, which is a better way to filter out popular books
popular_books['weighted popularity']=popular_books['Book-Rating']*popular_books['Total-Reviewers']

# Sorting them according to the weighted ratings
popular_books.sort_values('weighted popularity',ascending=False,inplace=True)

In [ ]:
popular_books.head(5)

In [ ]:
# dumping csv file 
popular_books['ISBN'].iloc[:30].to_csv('../data/processed/popular_books.csv',index=False)

In [ ]:
# we made a BookRecommender module, importing them
from utils.BookRecommender import recommend_popular_books

In [ ]:
# recommendation based on popularity
recommend_popular_books(5)

    2. Item-Based Collaborative Filtering (corelation )

In [ ]:
# importing necessary csv files
new_ratings_df = pd.read_csv("../data/processed/new_ratings_df.csv")
avg_rating_df = pd.read_csv("../data/processed/avg_rating_df.csv")

In [ ]:
display(new_ratings_df.sample(3))
avg_rating_df.sample(3)

In [ ]:
# We can merge new_ratings_df and  avg_rating_df on 'ISBN'
# also make a pivot table where all 'ISBN' are columns, 'User-ID' are rows and 'Book-Rating' are values

pivot_table = new_ratings_df.merge(avg_rating_df.iloc[:,[0,-1]],on='ISBN').pivot_table(index='User-ID',columns='ISBN',values='Book-Rating')

In [ ]:
pivot_table.sample(3)

In [ ]:
pivot_table.shape

In [ ]:
# will fill null values with 0
pivot_table.fillna(0,inplace=True)

In [ ]:
# due to memory issue, we will store all recommendations of each book in a dictionary
corr_rec_dict = dict()

Below command will run longer

In [ ]:
# for each book
# This will return correlation values with other books
# sorting them according to correlation values
# will store the index i.e ISBN into the dictionary

# for i in tqdm.tqdm(pivot_table.columns):
#     corr_isbns = tuple(pivot_table.corrwith(pivot_table[i]).sort_values(ascending=False).iloc[1:31].index)
#     corr_rec_dict[i] = corr_isbns

In [ ]:
# dumping the dictionary for later use
# joblib.dump(corr_rec_dict,"../data/processed/corr_rec_dict.joblib.z")

In [ ]:
# importing corr_recommend that we have created and stored in BookRecommender module
from utils.BookRecommender import  corr_recommend

In [ ]:
corr_recommend('0316666343')

    3. Content-Based Reccomendation System

In [ ]:
# Combining 'Book-Title','Book-Author','Year-Of-Publication' and 'Publisher' to variable

book_description = mer_rat_book.iloc[:,1]+' '+mer_rat_book.iloc[:,2]+' '+mer_rat_book.iloc[:,3].astype('str')+' '+mer_rat_book.iloc[:,4]

In [ ]:
# vectorize our words
tfidf=TfidfVectorizer(stop_words='english',ngram_range=(1,3))

In [ ]:
tfidf_matrix = tfidf.fit_transform(book_description)

In [ ]:
tfidf_matrix.shape

In [ ]:
# Now finding cisine similarity
cs_matrix = cosine_similarity(tfidf_matrix)

In [ ]:
#
cs_matrix.shape

In [ ]:
# # creating dict to recommend books
# cont_rec = dict()

# for idx in range(len(mer_rat_book)):
    
#     # sort them according to the cosine similarity
#     sort_index=sorted(list(enumerate(cs_matrix[idx])),key=lambda x:x[1],reverse=True) 
    
#     cont_rec[mer_rat_book.loc[idx,'ISBN']] = mer_rat_book.loc[[x[0] for x in sort_index[1:31]],'ISBN'].values

In [ ]:
# # dumping the dictionary
# joblib.dump(cont_rec,"../data/processed/cont_rec.joblib.z")

In [ ]:
# importing the recommendation engine 
from utils.BookRecommender import content_based_recommendation

In [ ]:
content_based_recommendation('0609804618')